In [1]:
import pandas as pd
import numpy as np

In [2]:
# This makes pandas display the full dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# This section is manipulating and recasting data

### Load data

In [3]:
hptrain = pd.read_csv("./train.csv")
hptest = pd.read_csv("./test.csv")

In [4]:
trainprice = hptrain["SalePrice"]
#testprice = hptest["SalePrice"] 
# This data is not included in the dataset and presumably needs to be uploaded to check for accuracy

In [5]:
hptrain.dtypes;

In [6]:
hptrain = hptrain.drop(["Id","SalePrice"],axis = 1);
hptest = hptest.drop(columns="Id",axis = 1)

### Recast MSSubClass as string type instead of numerical

In [7]:
hptrain = hptrain.astype({"MSSubClass":str})
hptest = hptest.astype({"MSSubClass":str})

### Prepare to encode some categorical variables as dummy variables

In [8]:
hptrncat = pd.get_dummies(hptrain[['MSSubClass','MSZoning','LandContour',"LotConfig","Neighborhood","Condition1",
                   "Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl", "Exterior1st",
                   "Exterior2nd","MasVnrType","Foundation","Heating","CentralAir","Electrical",
                   "GarageType","MiscFeature","SaleType","SaleCondition"]],dummy_na=True)
hptstcat = pd.get_dummies(hptest[['MSSubClass','MSZoning','LandContour',"LotConfig","Neighborhood","Condition1",
                   "Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl", "Exterior1st",
                   "Exterior2nd","MasVnrType","Foundation","Heating","CentralAir","Electrical",
                   "GarageType","MiscFeature","SaleType","SaleCondition"]],dummy_na=True)

### Ordinal encoding on other non-numeric variables

In [9]:
hptrain["Street"].value_counts(dropna=False) # I don't think "Street" will have much predictive power

Pave    1454
Grvl       6
Name: Street, dtype: int64

#### Determine which categorical variables have low variance (by eye)

In [10]:
hptrain.columns;

In [11]:
type(hptrain['MiscFeature'][0]) == str

False

In [12]:
hptrain['MiscFeature'].notna()[0]

False

In [13]:
for i in hptrain.columns:
    j=0
    while hptrain['MiscFeature'].isna()[j]:
        j+=1    
    if type(hptrain[i][j]) == str:
        vc = hptrain[i].value_counts(dropna=False)
        print(vc)

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64
Pave    1454
Grvl       6
Name: Street, dtype: int64
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64
AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64
Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     2

In [14]:
hptrain["Alley"].value_counts(dropna = False)

NaN     1369
Grvl      50
Pave      41
Name: Alley, dtype: int64

Remove: Street, Utilities.
Maybe remove: Condition2, RoofMat1, Heating, MiscFeature

In [15]:
hptrain = hptrain.drop(["Street","Utilities"],axis = 1)
hptest = hptest.drop(["Street","Utilities"],axis = 1)

In [16]:
test = hptrain['Fence'].replace(['GdPrv','MnPrv','GdWo','MnWw'],[0,1,2,3])

In [17]:
test=test.fillna(test.mean())

In [18]:
trnAlley = hptrain["Alley"].fillna(0).replace(['Grvl','Pave'],[1,2])
tstAlley = hptest["Alley"].fillna(0).replace(['Grvl','Pave'],[1,2])

trnLotShape = hptrain["LotShape"].replace(['Reg','IR1','IR2','IR3'],[0,1,2,3])
tstLotShape = hptest["LotShape"].replace(['Reg','IR1','IR2','IR3'],[0,1,2,3])

trnLandSlope = hptrain["LandSlope"].replace(['Gtl','Mod','Sev'],[0,1,2])
tstLandSlope = hptest["LandSlope"].replace(['Gtl','Mod','Sev'],[0,1,2])

trnExterQual = hptrain['ExterQual'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4]).astype(int)
tstExterQual = hptest['ExterQual'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4]).astype(int)

trnExterCond = hptrain['ExterCond'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
tstExterCond = hptest['ExterCond'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])

trnBsmtQual = hptrain['BsmtQual'].fillna(0).replace(['Ex','Gd','TA','Fa','Po'],[10,9,8,7,6]).astype(int)
tstBsmtQual = hptest['BsmtQual'].fillna(0).replace(['Ex','Gd','TA','Fa','Po'],[10,9,8,7,6]).astype(int)

trnBsmtCond = hptrain['BsmtCond'].fillna(2.5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
tstBsmtCond = hptest['BsmtCond'].fillna(2.5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
# unsure of the mapping here.
# It may be better to have no basement than a poor basement

trnBsmtExposure = hptrain['BsmtExposure'].fillna(3).replace(['Gd','Av','Mn','No'],[0,1,2,3])
tstBsmtExposure = hptest['BsmtExposure'].fillna(3).replace(['Gd','Av','Mn','No'],[0,1,2,3])
# What is the exposure level of no basement?

trnBsmtFinType1 = hptrain['BsmtFinType1'].fillna(5).replace(['GLQ','ALQ','BLQ','Rec','LwQ','Unf'],[0,1,2,3,4,5])
tstBsmtFinType1 = hptest['BsmtFinType1'].fillna(5).replace(['GLQ','ALQ','BLQ','Rec','LwQ','Unf'],[0,1,2,3,4,5])

trnBsmtFinType2 = hptrain['BsmtFinType2'].fillna(5).replace(['GLQ','ALQ','BLQ','Rec','LwQ','Unf'],[0,1,2,3,4,5])
tstBsmtFinType2 = hptest['BsmtFinType2'].fillna(5).replace(['GLQ','ALQ','BLQ','Rec','LwQ','Unf'],[0,1,2,3,4,5])

trnHeatingQC = hptrain['HeatingQC'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
tstHeatingQC = hptest['HeatingQC'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])

trnKitchenQual = hptrain['KitchenQual'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4]).astype(float)
tstKitchenQual = hptest['KitchenQual'].replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
tstKitchenQual = tstKitchenQual.fillna(tstKitchenQual.mean()).astype(float)

trnFunctional = hptrain['Functional'].replace(['Typ','Min1','Min2','Mod','Maj1','Maj2','Sev','Sal'],[0,1,2,3,4,5,6,7]).astype(float)
tstFunctional = hptest['Functional'].replace(['Typ','Min1','Min2','Mod','Maj1','Maj2','Sev','Sal'],[0,1,2,3,4,5,6,7])
tstFunctional = tstFunctional.fillna(tstFunctional.mean()).astype(float)

trnFireplaceQu = hptrain['FireplaceQu'].fillna(5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
tstFireplaceQu = hptest['FireplaceQu'].fillna(5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])

trnGarageFinish = hptrain['GarageFinish'].fillna(3).replace(['Fin','RFn','Unf'],[0,1,2])
tstGarageFinish = hptest['GarageFinish'].fillna(3).replace(['Fin','RFn','Unf'],[0,1,2])

trnGarageQual = hptrain['GarageQual'].fillna(5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
tstGarageQual = hptest['GarageQual'].fillna(5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])

trnGarageCond = hptrain['GarageCond'].fillna(5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
tstGarageCond = hptest['GarageCond'].fillna(5).replace(['Ex','Gd','TA','Fa','Po'],[0,1,2,3,4])
# This might be double penalizing houses without garages.
# I may have done something similar for basements above.

trnPavedDrive = hptrain['PavedDrive'].replace(['Y','P','N'],[0,1,2])
tstPavedDrive = hptest['PavedDrive'].replace(['Y','P','N'],[0,1,2])

trnPoolQC = hptrain['PoolQC'].fillna(4).replace(['Ex','Gd','TA','Fa'],[0,1,2,3]).astype(float)
tstPoolQC = hptest['PoolQC'].fillna(4).replace(['Ex','Gd','TA','Fa'],[0,1,2,3])
tstPoolQC = tstPoolQC.fillna(tstPoolQC.mean()).astype(float)

trnFence = hptrain['Fence'].replace(['GdPrv','MnPrv','GdWo','MnWw'],[0,1,2,3])
trnFence = trnFence.fillna(trnFence.mean())
tstFence = hptest['Fence'].replace(['GdPrv','MnPrv','GdWo','MnWw'],[0,1,2,3])
tstFence = tstFence.fillna(trnFence.mean())

In [19]:
tstFunctional.value_counts(dropna = 0)

0.000000    1357
2.000000      36
1.000000      34
3.000000      20
4.000000       5
5.000000       4
0.145504       2
6.000000       1
Name: Functional, dtype: int64

## Concatenating all data together

In [20]:
hpXtr = pd.concat([trnAlley,trnLotShape,trnLandSlope,trnExterQual,trnExterCond,trnBsmtQual,
                   trnBsmtCond,trnBsmtExposure,trnBsmtFinType1,trnBsmtFinType2,trnHeatingQC,
                   trnKitchenQual,trnFunctional,trnFireplaceQu,trnGarageFinish,trnGarageQual,
                   trnGarageCond,trnPavedDrive,trnPoolQC,trnFence],axis = 1)
hpXte = pd.concat([tstAlley,tstLotShape,tstLandSlope,tstExterQual,tstExterCond,tstBsmtQual,
                   tstBsmtCond,tstBsmtExposure,tstBsmtFinType1,tstBsmtFinType2,tstHeatingQC,
                   tstKitchenQual,tstFunctional,tstFireplaceQu,tstGarageFinish,tstGarageQual,
                   tstGarageCond,tstPavedDrive,tstPoolQC,tstFence],axis = 1)

In [21]:
hpXtr = pd.concat([hpXtr,hptrncat],axis = 1)
hpXte = pd.concat([hpXte,hptstcat],axis = 1)

In [22]:
hptrainN = hptrain.drop(['MSSubClass','MSZoning','LandContour',"LotConfig","Neighborhood","Condition1",
                   "Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl", "Exterior1st",
                   "Exterior2nd","MasVnrType","Foundation","Heating","CentralAir","Electrical",
                   "GarageType","MiscFeature","SaleType","SaleCondition",'Alley','LotShape','LandSlope',
                   'ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC',
                   'KitchenQual','Functional','FireplaceQu','GarageFinish','GarageQual','GarageCond',
                   'PavedDrive','PoolQC','Fence'],axis = 1);
hptestN = hptrain.drop(['MSSubClass','MSZoning','LandContour',"LotConfig","Neighborhood","Condition1",
                   "Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl", "Exterior1st",
                   "Exterior2nd","MasVnrType","Foundation","Heating","CentralAir","Electrical",
                   "GarageType","MiscFeature","SaleType","SaleCondition",'Alley','LotShape','LandSlope',
                   'ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC',
                   'KitchenQual','Functional','FireplaceQu','GarageFinish','GarageQual','GarageCond',
                   'PavedDrive','PoolQC','Fence'],axis = 1);

In [23]:
hpXtr = pd.concat([hpXtr,hptrainN],axis = 1)
hpXte = pd.concat([hpXte,hptestN],axis = 1)

In [32]:
hpXtr.dtypes.size

254

In [33]:
hpXte.dtypes.size

240

In [37]:
set(hpXtr.columns).difference(set(hpXte.columns))

{'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'Electrical_Mix',
 'Exterior1st_ImStucc',
 'Exterior1st_Stone',
 'Exterior2nd_Other',
 'Heating_Floor',
 'Heating_OthW',
 'HouseStyle_2.5Fin',
 'MiscFeature_TenC',
 'RoofMatl_ClyTile',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll'}

In [38]:
set(hpXte.columns).difference(set(hpXtr.columns))

{'MSSubClass_150'}